In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F
from torchvision import models
from tqdm import tqdm

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Using device:", device)


Using device: cuda


In [2]:
# --- Data Transforms ---
mean = (0.4914, 0.4822, 0.4465)
std  = (0.2023, 0.1994, 0.2010)

transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean, std),
])
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean, std),
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [3]:
# --- Defense Modules ---
class ThresholdReLU(nn.Module):
    def __init__(self, threshold=0.001):
        super().__init__()
        self.threshold = threshold
    def forward(self, x):
        return torch.where(x > self.threshold, x, torch.zeros_like(x))

def quantize_input(x, levels=16, min_val=-2.5, max_val=2.5):
    x = torch.clamp(x, min_val, max_val)
    x_norm = (x - min_val) / (max_val - min_val)
    x_quant = torch.round(x_norm * (levels - 1)) / (levels - 1)
    return x_quant * (max_val - min_val) + min_val

In [4]:
# --- Sparsity Loss ---
def sparsity_loss_modified(activations, beta=20.0):
    total = sum(act.numel() for act in activations)
    loss = 0.0
    for act in activations:
        loss += torch.sum(1.0 - torch.tanh(beta * torch.abs(act)))
    return loss / total

In [5]:
# --- VGG16 with Sparsity Recording ---
class VGG16Sparse(nn.Module):
    def __init__(self, num_classes=10):
        super().__init__()
        base = models.vgg16(pretrained=True)
        # Adapt classifier for CIFAR-10
        base.classifier[6] = nn.Linear(4096, num_classes)
        # Replace ReLU in features
        self.features = nn.Sequential(*[
            (ThresholdReLU() if isinstance(m, nn.ReLU) else m)
            for m in base.features
        ])
        self.avgpool = base.avgpool
        self.classifier = base.classifier
    def forward(self, x):
        activations = []
        for layer in self.features:
            x = layer(x)
            if isinstance(layer, ThresholdReLU):
                activations.append(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x, activations

In [6]:
# --- Training/Evaluation ---
def train_one_epoch(model, loader, criterion, optimizer):
    model.train()
    total_loss, correct, total = 0, 0, 0
    for imgs, labels in loader:
        imgs, labels = imgs.to(device), labels.to(device)
        imgs = quantize_input(imgs)
        outputs, activations = model(imgs)
        loss_ce = criterion(outputs, labels)
        loss_sp = sparsity_loss_modified(activations)
        loss = loss_ce + loss_sp
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * imgs.size(0)
        _, preds = outputs.max(1)
        correct += preds.eq(labels).sum().item()
        total += labels.size(0)
    return total_loss / total, 100 * correct / total

In [7]:
def evaluate(model, loader, criterion):
    model.eval()
    total_loss, correct, total = 0, 0, 0
    with torch.no_grad():
        for imgs, labels in loader:
            imgs, labels = imgs.to(device), labels.to(device)
            imgs = quantize_input(imgs)
            outputs, activations = model(imgs)
            loss_ce = criterion(outputs, labels)
            loss_sp = sparsity_loss_modified(activations)
            loss = loss_ce + loss_sp
            total_loss += loss.item() * imgs.size(0)
            _, preds = outputs.max(1)
            correct += preds.eq(labels).sum().item()
            total += labels.size(0)
    return total_loss / total, 100 * correct / total

In [8]:
# --- White-box Sparsity Attack ---
def generate_sparsity_adversary(model, x_clean, y_clean, criterion,
                                epsilon=0.3, alpha=0.01, num_iter=75, c=5.0, beta=20.0):
    model.eval()
    x_adv = x_clean.clone().detach().to(device)
    x_adv.requires_grad = True
    for i in range(num_iter):
        outputs, activations = model(x_adv)
        loss_ce = criterion(outputs, y_clean)
        loss_sp = sparsity_loss_modified(activations, beta=beta)
        loss = loss_sp + c * loss_ce
        model.zero_grad()
        loss.backward()
        x_adv = x_adv - alpha * x_adv.grad.sign()
        x_adv = torch.max(torch.min(x_adv, x_clean + epsilon), x_clean - epsilon)
        x_adv = torch.clamp(x_adv, -2.5, 2.5)
        x_adv = x_adv.detach()
        x_adv.requires_grad = True
        if (i + 1) % 15 == 0:
            print(f"Iter {i+1}/{num_iter}, CE: {loss_ce.item():.4f}, SP: {loss_sp.item():.4f}")
    return x_adv.detach()

In [9]:
# --- Instantiate, Train/Load, and Test Attack ---
model = VGG16Sparse().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=1e-4)

/home/rushik/.local/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/rushik/.local/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [10]:
model.eval()
for imgs, labels in testloader:
    imgs, labels = imgs.to(device), labels.to(device)
    imgs_q = quantize_input(imgs)
    out_c, acts_c = model(imgs_q)
    _, p_c = out_c.max(1)
    imgs_adv = generate_sparsity_adversary(model, imgs_q, p_c, criterion)
    out_a, acts_a = model(imgs_adv)
    _, p_a = out_a.max(1)
    print("Clean preds:", p_c[:10])
    print("Adv preds: ", p_a[:10])
    spar_c = sum((act != 0).float().sum() for act in acts_c) / sum(act.numel() for act in acts_c)
    spar_a = sum((act != 0).float().sum() for act in acts_a) / sum(act.numel() for act in acts_a)
    print(f"Sparsity clean: {spar_c:.4f}, adv: {spar_a:.4f}")
    break

Iter 15/75, CE: 0.2557, SP: 0.6078
Iter 30/75, CE: 0.1067, SP: 0.6073
Iter 45/75, CE: 0.0557, SP: 0.6063
Iter 60/75, CE: 0.0351, SP: 0.6054
Iter 75/75, CE: 0.0259, SP: 0.6044
Clean preds: tensor([7, 7, 3, 0, 5, 3, 7, 5, 2, 7], device='cuda:0')
Adv preds:  tensor([7, 7, 3, 0, 5, 3, 7, 5, 2, 7], device='cuda:0')
Sparsity clean: 0.3980, adv: 0.4013
